In [18]:
# get the average velocity from the ground truth data

import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import pandas as pd
import numpy as np

SIMDATA_PATH="attack_data/sim_data/mules/"
GTDATA_PATH ="attack_data/ground_truth/"

def average_velocity_from_trace(trace_file, limit=10000000, downsample=1):
    
    print("extracting average velocity from", trace_file)
    
    # import file
    gateway_df = pd.read_csv(trace_file)

    # down sample to every 100
    gateway_df_smol = gateway_df[0:limit:downsample]
    gateway_df_smol.head()
    
    # filter to adjadent gateways with a time difference larger than 3 seconds
#     diff_gateway_df = gateway_df_smol.diff()
    
#     print(gateway_df_smol)
    
#     backbone_gateway_df = gateway_df_smol[diff_gateway_df['norm_secs (secs)'] > 1]
#     backbone_gateway_df.head()

    # import the ground truth gateway location file
    gateway_positions_df = pd.read_csv(GTDATA_PATH + 'sensors.csv')
    
    if gateway_df_smol.shape[0] <= 1:
        print('  only', gateway_df_smol.shape[0], 'points, aborting')
        return

    velocities = []
    for i in tqdm(range(1, gateway_df_smol.shape[0]), position=0, leave=True):
        
        print(gateway_df_smol.iloc[0][' norm_secs (secs)'])

        src_gateway_id, dst_gateway_id = int(gateway_df_smol.iloc[i-1]['sensor ID']), int(gateway_df_smol.iloc[i]['sensor ID'])
        src_time_s, dst_time_s = float(gateway_df_smol.iloc[i-1]['norm_secs (secs)']), float(gateway_df_smol.iloc[i]['norm_secs (secs)'])

        src_gateway_x, src_gateway_y = gateway_positions_df.iloc[src_gateway_id]['x (m)'], gateway_positions_df.iloc[src_gateway_id]['y (m)']
        dst_gateway_x, dst_gateway_y = gateway_positions_df.iloc[dst_gateway_id]['x (m)'], gateway_positions_df.iloc[dst_gateway_id]['y (m)']

        distance = math.sqrt((src_gateway_x - dst_gateway_x)**2 + (src_gateway_y - dst_gateway_y)**2)
        if distance == 0 and src_gateway_id == dst_gateway_id:
            continue
#         if distance == 0:
#             continue
        velocities.append(
            distance / (dst_time_s - src_time_s)
        )

    _ = plt.hist(velocities, range=[0, 10], bins='auto')
    plt.show()

    q75, q25 = np.percentile(velocities, [75 ,25])
    iqr = q75 - q25

    print('q25', q25, 'q50', q25 + iqr/2, 'q75', q75)
    print()
    
    
# average_velocity_from_trace(SIMDATA_PATH + "104.csv", limit=100000000)
# average_velocity_from_trace(SIMDATA_PATH + "135.csv", limit=100000000)

SMOLDATA_PATH = "../smol_attacks/smol_sim/mules/"
mule_ids = ['007', '011', '022', '024', '025', '038', '052', '062', '071', '085', '126', '167']

for m in mule_ids:
    average_velocity_from_trace(SMOLDATA_PATH+m+'.csv', downsample=1)


extracting average velocity from ../smol_attacks/smol_sim/mules/007.csv


  0%|          | 0/376 [00:00<?, ?it/s]

63.71550293847541


KeyError: 'norm_secs (secs)'